## 2. Импорт библиотек и настройка

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Скачиваем стоп-слова для русского языка
nltk.download('stopwords')
nltk.download('punkt')

print("Библиотеки успешно импортированы!")


Библиотеки успешно импортированы!


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZubarevVV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ZubarevVV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3. Загрузка и предварительный анализ данных

In [2]:
# Загрузка данных
df = pd.read_excel('training_tmc.xlsx')

# Посмотрим на структуру данных
print("Размер данных:", df.shape)
print("\nПервые 5 строк:")
print(df.head())

print("\nИнформация о данных:")
print(df.info())

# Посмотрим на распределение классов
print("\nРаспределение по классам:")
print(df['Hierarchy_MTR_Name'].value_counts())


Размер данных: (104945, 12)

Первые 5 строк:
   Hierarchy_MTR_Class                        Hierarchy_MTR_Name  CSCD_ID  \
0     4002011102020000  Материалы смазочные мелк.,средн. фасовка  3648365   
1     4002011102020000  Материалы смазочные мелк.,средн. фасовка  3648364   
2     3001010101010200                    Куртки лет муж ЛМ01-01  6087431   
3     3001010101010200                    Куртки лет муж ЛМ01-01  6087428   
4     3001010101010200                    Куртки лет муж ЛМ01-01  6087429   

                   SHORT_NAME/ru_RU  AUTO_SHORT_NAME  \
0  Масло L GEAR GL-4 SYN 75W85 к.4л              NaN   
1  Масло L GEAR GL-4 SYN 75W85 к.1л              NaN   
2                  Куртка ЛМ01-01 2              NaN   
3  Куртка ЛМ01-01 2 104-108/158-164              NaN   
4   Куртка ЛМ01-01 2 96-100/170-176              NaN   

                                     FULL_NAME/ru_RU  AUTO_FULL_NAME  \
0  Масло трансмиссионное LUKOIL GEAR GL-4 SYNTH 7...             NaN   
1  Масло тр

## 4. Предобработка текстовых данных

In [3]:
# Функция для предобработки текста
def preprocess_text(text):
    if pd.isna(text):
        return ""
    
    # Приводим к нижнему регистру
    text = text.lower()
    
    # Удаляем специальные символы, но сохраняем числа и размеры (например, 30х3,4)
    text = re.sub(r'[^\w\s\dх×]', ' ', text)
    
    # Заменяем различные варианты написания 'х' на стандартный
    text = re.sub(r'[х×]', ' x ', text)
    
    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Применяем предобработку к текстовым полям
df['processed_short_name'] = df['SHORT_NAME/ru_RU'].apply(preprocess_text)
df['processed_full_name'] = df['FULL_NAME/ru_RU'].apply(preprocess_text)

# Объединяем короткое и полное название для лучшего контекста
df['combined_text'] = df['processed_short_name'] + ' ' + df['processed_full_name']

# Посмотрим на результат предобработки
print("Примеры обработанных текстов:")
for i in range(3):
    print(f"Оригинал: {df['SHORT_NAME/ru_RU'].iloc[i]}")
    print(f"Обработанный: {df['combined_text'].iloc[i]}")
    print("-" * 50)


Примеры обработанных текстов:
Оригинал: Масло L GEAR GL-4 SYN 75W85 к.4л
Обработанный: масло l gear gl 4 syn 75w85 к 4л масло трансмиссионное lukoil gear gl 4 synth 75w 85 канистра п э 4л
--------------------------------------------------
Оригинал: Масло L GEAR GL-4 SYN 75W85 к.1л
Обработанный: масло l gear gl 4 syn 75w85 к 1л масло трансмиссионное lukoil gear gl 4 synth 75w 85 канистра п э 1л
--------------------------------------------------
Оригинал: Куртка ЛМ01-01 2
Обработанный: куртка лм01 01 2 куртка летняя мужская для защиты от нефти и нефтепродуктов лм01 01 группа ткани 2 сто лукойл 1 6 15 1 1 2023
--------------------------------------------------


## 5. Подготовка данных для модели

In [4]:
# Кодируем целевые метки
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['Hierarchy_MTR_Name'])

# Сохраняем mapping для обратного преобразования
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Соответствие меток:")
for label, encoded in label_mapping.items():
    print(f"{encoded}: {label}")

# Подготовка текстовых данных
texts = df['combined_text'].tolist()
labels = df['encoded_labels'].values

# Параметры токенизации
MAX_NB_WORDS = 10000  # Максимальное количество слов в словаре
MAX_SEQUENCE_LENGTH = 100  # Максимальная длина последовательности
EMBEDDING_DIM = 100  # Размерность векторного представления

# Токенизация текстов
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='', lower=False)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Паддинг последовательностей до одинаковой длины
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Подготовка меток
y = to_categorical(labels)
num_classes = y.shape[1]

print(f"Размерность данных: {X.shape}")
print(f"Количество классов: {num_classes}")
print(f"Размер словаря: {len(tokenizer.word_index)}")


Соответствие меток:
0: 3аготовки трубные
1: DIN-рейки
2: USB-Токены
3: [На Блок] Отводы для трубопроводов без покрытия
4: [На Блок] Тройники для трубопроводов без покрытия
5: [На Блок] пластификаторы
6: Абразивы
7: Автобусы
8: Автомобили грузовые
9: Автомобили легковые
10: Автомобили пожарные
11: Автохимические жидкости (не нефтехимия)
12: Адаптеры (перех.) трубопр. по чертежу
13: Адаптеры (перех.) трубопр. стандартные
14: Адаптеры (перех.) трубопр. фирменные
15: Адаптеры для электросети
16: Адаптеры компьютерные
17: Аксессуары компьютерные
18: Анализаторы лабораторные
19: Анкеры
20: Антенны
21: Аппаратура для контроля скважин
22: Аппараты воздушного охлаждения (АВО)
23: Аппараты емкостные вертикал. ВПП
24: Аппараты емкостные вертикал. ВЭЭ
25: Аппараты емкостные вертикал. прочие
26: Аппараты емкостные горизонт. ГЭЭ
27: Аппараты емкостные горизонт. прочие
28: Аппараты и установки сушильные
29: Аппараты телефонные (телефоны)
30: Арматура для ВЛ
31: Арматура к приборам контроля давления
3

## 6. Анализ распределения классов и решение проблемы малых классов

In [5]:
# Анализируем распределение классов
class_distribution = df['Hierarchy_MTR_Name'].value_counts()
print("Распределение классов:")
print(class_distribution)

# Определяем порог для малых классов
MIN_SAMPLES_PER_CLASS = 2  # Минимальное количество примеров для стратификации

# Находим классы с достаточным количеством примеров
valid_classes = class_distribution[class_distribution >= MIN_SAMPLES_PER_CLASS].index
small_classes = class_distribution[class_distribution < MIN_SAMPLES_PER_CLASS].index

print(f"\nКлассы с достаточным количеством данных ({len(valid_classes)}):")
print(valid_classes.tolist())

print(f"\nМелкие классы (будут исключены из обучения) ({len(small_classes)}):")
print(small_classes.tolist())

# Фильтруем данные, оставляя только классы с достаточным количеством примеров
df_filtered = df[df['Hierarchy_MTR_Name'].isin(valid_classes)].copy()

print(f"\nИсходный размер датасета: {len(df)}")
print(f"Размер после фильтрации: {len(df_filtered)}")
print(f"Удалено записей: {len(df) - len(df_filtered)}")

# Перекодируем метки для отфильтрованных данных
label_encoder = LabelEncoder()
df_filtered['encoded_labels'] = label_encoder.fit_transform(df_filtered['Hierarchy_MTR_Name'])

# Сохраняем mapping для обратного преобразования
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("\nСоответствие меток после фильтрации:")
for label, encoded in label_mapping.items():
    print(f"{encoded}: {label} (примеров: {class_distribution[label]})")


Распределение классов:
Hierarchy_MTR_Name
Фильтры автомобильные                       8579
Узлы и компл. д/нест. оборуд. по чертежу    6057
Запчасти и узлы автомобил.трансп.прочие     5152
Узлы, детали оборуд. насосного прочие       4821
Узлы и компл. д/нест. оборуд. фирменные     3665
                                            ... 
Полиспасты                                     1
ПС сканирования и распознавания док.           1
Полуботинки дем. муж. НмСжСмЭс                 1
Хомуты автомобильные                           1
Отстойники нефти ОГ                            1
Name: count, Length: 1645, dtype: int64

Классы с достаточным количеством данных (1502):
['Фильтры автомобильные', 'Узлы и компл. д/нест. оборуд. по чертежу', 'Запчасти и узлы автомобил.трансп.прочие', 'Узлы, детали оборуд. насосного прочие', 'Узлы и компл. д/нест. оборуд. фирменные', 'Изделия и детали изготавливаемые', 'Узлы, детали оборуд. компрес. прочие', 'Узлы, детали оборуд. насос. по чертежу', 'Насосы скважин

## 7. Подготовка текстовых данных для отфильтрованного датасета

In [6]:
# Подготовка текстовых данных для отфильтрованного датасета
texts = df_filtered['combined_text'].tolist()
labels = df_filtered['encoded_labels'].values

# Параметры токенизации (можно уменьшить для меньшего датасета)
MAX_NB_WORDS = 5000  # Уменьшаем размер словаря для меньшего датасета
MAX_SEQUENCE_LENGTH = 50   # Уменьшаем длину последовательности
EMBEDDING_DIM = 100

# Токенизация текстов
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='', lower=False)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Паддинг последовательностей до одинаковой длины
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Подготовка меток
y = to_categorical(labels)
num_classes = y.shape[1]

print(f"Размерность данных: {X.shape}")
print(f"Количество классов: {num_classes}")
print(f"Размер словаря: {len(tokenizer.word_index)}")
print(f"Минимальное количество примеров в классе: {np.min(np.sum(y, axis=0))}")


Размерность данных: (104802, 50)
Количество классов: 1502
Размер словаря: 77892
Минимальное количество примеров в классе: 2.0


## 8. Стратифицированное разделение данных с проверкой

In [7]:
# Дополнительное уменьшение параметров для экономии памяти
MAX_NB_WORDS = 3000   # Еще больше уменьшаем словарь
MAX_SEQUENCE_LENGTH = 30    # Еще больше уменьшаем длину последовательности

# Пересоздаем X с новыми параметрами
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(f"Новая размерность данных: {X.shape}")


Новая размерность данных: (104802, 30)


In [8]:
def simple_split(X, y, test_size=0.2, val_size=0.2, random_state=42):
    """
    Простое разделение без стратификации (менее оптимально, но надежно)
    """
    # Сначала разделяем на train и temp
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=(test_size + val_size), random_state=random_state
    )
    
    # Затем разделяем temp на val и test
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=test_size/(test_size + val_size), random_state=random_state
    )
    
    print(f"Размеры выборок:")
    print(f"Обучающая: {X_train.shape[0]} примеров")
    print(f"Валидационная: {X_val.shape[0]} примеров")
    print(f"Тестовая: {X_test.shape[0]} примеров")
    
    # Проверяем распределение классов
    print("\nРаспределение классов (предупреждение: без стратификации):")
    train_classes = np.unique(np.argmax(y_train, axis=1))
    val_classes = np.unique(np.argmax(y_val, axis=1))
    test_classes = np.unique(np.argmax(y_test, axis=1))
    
    print(f"Обучающая: {len(train_classes)} классов")
    print(f"Валидационная: {len(val_classes)} классов") 
    print(f"Тестовая: {len(test_classes)} классов")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Используем простой метод если другие не работают
X_train, X_val, X_test, y_train, y_val, y_test = simple_split(X, y)





Размеры выборок:
Обучающая: 62881 примеров
Валидационная: 20960 примеров
Тестовая: 20961 примеров

Распределение классов (предупреждение: без стратификации):
Обучающая: 1447 классов
Валидационная: 1153 классов
Тестовая: 1164 классов


## 9. Альтернативное решение: увеличение веса малых классов

In [9]:
# Если после фильтрации остались классы с малым количеством примеров, 
# можно использовать взвешивание классов

from sklearn.utils.class_weight import compute_class_weight

# Вычисляем веса классов для компенсации дисбаланса
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(labels),
    y=labels
)

# Создаем словарь весов для Keras
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("Веса классов для компенсации дисбаланса:")
for cls, weight in class_weight_dict.items():
    class_name = label_encoder.inverse_transform([cls])[0]
    print(f"  {class_name}: {weight:.2f}")


Веса классов для компенсации дисбаланса:
  3аготовки трубные: 34.89
  DIN-рейки: 7.75
  USB-Токены: 8.72
  [На Блок] Отводы для трубопроводов без покрытия: 13.95
  [На Блок] Тройники для трубопроводов без покрытия: 23.26
  Абразивы: 5.37
  Автомобили легковые: 17.44
  Автомобили пожарные: 4.98
  Автохимические жидкости (не нефтехимия): 34.89
  Адаптеры (перех.) трубопр. по чертежу: 11.63
  Адаптеры (перех.) трубопр. стандартные: 34.89
  Адаптеры (перех.) трубопр. фирменные: 1.04
  Адаптеры для электросети: 17.44
  Адаптеры компьютерные: 4.10
  Аксессуары компьютерные: 0.34
  Анализаторы лабораторные: 1.34
  Анкеры: 8.72
  Антенны: 2.33
  Аппаратура для контроля скважин: 4.98
  Аппараты воздушного охлаждения (АВО): 11.63
  Аппараты емкостные вертикал. ВЭЭ: 17.44
  Аппараты емкостные горизонт. ГЭЭ: 23.26
  Аппараты емкостные горизонт. прочие: 6.34
  Аппараты и установки сушильные: 4.36
  Аппараты телефонные (телефоны): 0.66
  Арматура для ВЛ: 23.26
  Арматура к приборам контроля давления

## 10. Создание модели CNN + LSTM

In [14]:
def create_cnn_lstm_model_fixed(vocab_size, embedding_dim, max_sequence_length, num_classes):
    model = Sequential()
    
    # Слой Embedding - исправляем input_dim
    model.add(Embedding(
        input_dim=vocab_size + 1,  # +1 потому что индексация с 0
        output_dim=embedding_dim,
        input_length=max_sequence_length,
        name='embedding'
    ))
    
    # Сверточные слои
    model.add(Conv1D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    
    model.add(Conv1D(32, 3, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    
    # Рекуррентный слой
    model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.1))
    
    # Полносвязные слои
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation='relu'))
    
    # Выходной слой
    model.add(Dense(num_classes, activation='softmax'))
    
    # Компиляция модели
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Проверяем размерности данных
print("Проверка размерностей данных:")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"Vocabulary size: {vocab_size}")
print(f"Number of classes: {num_classes}")

# Проверяем диапазон значений в X_train
print(f"\nДиапазон значений в X_train: от {np.min(X_train)} до {np.max(X_train)}")

# Создаем модель с правильными параметрами
model = create_cnn_lstm_model_fixed(vocab_size, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, num_classes)

print("\nАрхитектура модели:")
model.summary()

Проверка размерностей данных:
X_train shape: (62881, 30)
y_train shape: (62881, 1502)
X_val shape: (20960, 30)
y_val shape: (20960, 1502)
Vocabulary size: 3000
Number of classes: 1502

Диапазон значений в X_train: от 0 до 4999

Архитектура модели:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Если предыдущая модель не работает, создадим более простую
def create_simple_model(vocab_size, embedding_dim, max_sequence_length, num_classes):
    model = Sequential()
    
    # Более простой embedding слой
    model.add(Embedding(
        input_dim=vocab_size + 1,
        output_dim=embedding_dim,
        input_length=max_sequence_length,
        mask_zero=True  # Игнорировать нулевые padding значения
    ))
    
    # Глобальный пулинг вместо сверток
    model.add(GlobalMaxPooling1D())
    
    # Всего один скрытый слой
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    
    # Выходной слой
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Пробуем создать упрощенную модель
print("Создаем упрощенную модель...")
simple_model = create_simple_model(vocab_size, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, num_classes)
simple_model.summary()


Создаем упрощенную модель...


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 11. Обучение модели с учетом дисбаланса (измененный блок обучения)

In [21]:
# Дополнительная проверка данных
def validate_data(X_train, y_train, X_val, y_val, tokenizer):
    """Проверка целостности данных перед обучением"""
    
    # Проверяем, что все индексы в пределах словаря
    max_index = np.max(X_train)
    vocab_size_actual = len(tokenizer.word_index) + 1  # +1 для padding
    print(f"Максимальный индекс в данных: {max_index}")
    print(f"Размер словаря: {vocab_size_actual}")
    
    if max_index >= vocab_size_actual:
        print(f"ОШИБКА: Найден индекс {max_index}, но размер словаря только {vocab_size_actual}")
        # Исправляем проблему
        X_train = np.clip(X_train, 0, vocab_size_actual - 1)
        X_val = np.clip(X_val, 0, vocab_size_actual - 1)
        print("Данные исправлены через clipping")
    
    # Проверяем соответствие размерностей
    assert X_train.shape[0] == y_train.shape[0], "Несоответствие размеров X_train и y_train"
    assert X_val.shape[0] == y_val.shape[0], "Несоответствие размеров X_val и y_val"
    
    # Проверяем, что y содержит вероятности
    y_sum = np.sum(y_train, axis=1)
    print(f"Сумма по строкам y_train: min={np.min(y_sum):.2f}, max={np.max(y_sum):.2f}")
    
    return X_train, X_val

# Проверяем и исправляем данные если нужно
X_train, X_val = validate_data(X_train, y_train, X_val, y_val, tokenizer)


Максимальный индекс в данных: 4999
Размер словаря: 77893
Сумма по строкам y_train: min=1.00, max=1.00


In [20]:
# Callback для ранней остановки
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Функция для безопасного обучения
def safe_model_fit(model, X_train, y_train, X_val, y_val, class_weight_dict=None):
    try:
        print("Начинаем обучение модели...")
        
        history = model.fit(
            X_train, y_train,
            batch_size=16,
            epochs=50,  # Уменьшаем количество эпох для теста
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            class_weight=class_weight_dict,
            verbose=1
        )
        return history, True
    except Exception as e:
        print(f"Ошибка при обучении: {e}")
        print("Пробуем альтернативные настройки...")
        return None, False

# Пробуем обучить упрощенную модель сначала
print("=== Тестируем упрощенную модель ===")
history_simple, success_simple = safe_model_fit(
    simple_model, X_train, y_train, X_val, y_val, class_weight_dict
)

if success_simple:
    print("Упрощенная модель успешно обучена!")
    model = simple_model  # Используем упрощенную модель
    history = history_simple
else:
    print("Пробуем оригинальную модель...")
    # Пробуем оригинальную модель с меньшим batch size
    try:
        history = model.fit(
            X_train, y_train,
            batch_size=8,  # Еще меньше batch size
            epochs=50,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            class_weight=class_weight_dict,
            verbose=1
        )
    except Exception as e:
        print(f"Ошибка с оригинальной моделью: {e}")
        print("Создаем минимальную модель...")
        
        # Создаем минимальную модель
        minimal_model = Sequential([
            Embedding(vocab_size + 1, 50, input_length=MAX_SEQUENCE_LENGTH),
            GlobalMaxPooling1D(),
            Dense(16, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
        minimal_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        history = minimal_model.fit(
            X_train, y_train,
            batch_size=8,
            epochs=50,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=1
        )
        model = minimal_model

print("Обучение завершено!")


=== Тестируем упрощенную модель ===
Начинаем обучение модели...
Epoch 1/50
Ошибка при обучении: Graph execution error:

Detected at node sequential_3_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 640, in run_forever

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Pyt

InvalidArgumentError: Graph execution error:

Detected at node sequential_5_1/embedding_2_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 640, in run_forever

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1992, in _run_once

  File "c:\

## 12. Визуализация процесса обучения

In [19]:
# Функция для построения графиков обучения
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # График точности
    ax1.plot(history.history['accuracy'], label='Обучающая точность')
    ax1.plot(history.history['val_accuracy'], label='Валидационная точность')
    ax1.set_title('Точность модели')
    ax1.set_xlabel('Эпоха')
    ax1.set_ylabel('Точность')
    ax1.legend()
    ax1.grid(True)
    
    # График потерь
    ax2.plot(history.history['loss'], label='Обучающие потери')
    ax2.plot(history.history['val_loss'], label='Валидационные потери')
    ax2.set_title('Потери модели')
    ax2.set_xlabel('Эпоха')
    ax2.set_ylabel('Потери')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

# Строим графики
plot_training_history(history)


NameError: name 'history' is not defined

## 13. Оценка модели

In [ ]:
# Оценка на тестовых данных
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Точность на тестовых данных: {test_accuracy:.4f}")

# Предсказания
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Отчет о классификации
print("\nОтчет о классификации:")
print(classification_report(
    y_true_classes, 
    y_pred_classes,
    target_names=label_encoder.classes_
))

# Матрица ошибок
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true_classes, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.title('Матрица ошибок')
plt.xlabel('Предсказанные метки')
plt.ylabel('Истинные метки')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 14. Функция для предсказания на новых данных

In [ ]:
def predict_category(text, model, tokenizer, label_encoder, max_sequence_length):
    """
    Функция для предсказания категории нового текста
    """
    # Предобработка текста
    processed_text = preprocess_text(text)
    
    # Токенизация и паддинг
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # Предсказание
    prediction = model.predict(padded_sequence, verbose=0)
    predicted_class_idx = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    # Преобразование обратно в текстовую метку
    predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
    
    return predicted_class, confidence

# Тестирование функции на примерах
test_texts = [
    "Масло трансмиссионное синтетическое 75W90",
    "Труба полипропиленовая PPR 20мм",
    "Куртка летняя защитная",
    "Теплообменник стальной"
]

print("Тестирование предсказаний:")
print("=" * 60)
for text in test_texts:
    category, confidence = predict_category(text, model, tokenizer, label_encoder, MAX_SEQUENCE_LENGTH)
    print(f"Текст: '{text}'")
    print(f"Предсказанная категория: {category}")
    print(f"Уверенность: {confidence:.4f}")
    print("-" * 40)


## 15 Сохранение модели и компонентов

In [ ]:
import pickle
import json

# Сохраняем модель
model.save('tmc_classification_model.h5')

# Сохраняем токенизатор
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Сохраняем label encoder
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Сохраняем параметры
model_params = {
    'MAX_NB_WORDS': MAX_NB_WORDS,
    'MAX_SEQUENCE_LENGTH': MAX_SEQUENCE_LENGTH,
    'EMBEDDING_DIM': EMBEDDING_DIM
}

with open('model_params.json', 'w', encoding='utf-8') as f:
    json.dump(model_params, f, ensure_ascii=False, indent=2)

print("Модель и компоненты успешно сохранены!")


## ОТЛАДКА

In [25]:
# Перепроверим количество классов
print("=== АНАЛИЗ ПРОБЛЕМЫ ===")
print(f"Количество классов: {num_classes}")
print(f"Размер y_train: {y_train.shape}")
print(f"Количество примеров в обучающей выборке: {X_train.shape[0]}")

# Проверим, действительно ли у нас 1502 класса
actual_classes = len(np.unique(np.argmax(y_train, axis=1)))
print(f"Фактическое количество классов в данных: {actual_classes}")

# Проблема: слишком много классов для маленького датасета
# Решение: объединим мелкие классы или возьмем только основные


=== АНАЛИЗ ПРОБЛЕМЫ ===
Количество классов: 1502
Размер y_train: (62881, 1502)
Количество примеров в обучающей выборке: 62881
Фактическое количество классов в данных: 1447


In [26]:
# Оставляем только классы с достаточным количеством примеров
def filter_top_classes(X, y, min_samples_per_class=5):
    """Оставляем только классы с минимальным количеством примеров"""
    y_labels = np.argmax(y, axis=1)
    unique, counts = np.unique(y_labels, return_counts=True)
    
    # Находим классы с достаточным количеством примеров
    valid_classes = unique[counts >= min_samples_per_class]
    
    print(f"Всего классов: {len(unique)}")
    print(f"Классы с >= {min_samples_per_class} примерами: {len(valid_classes)}")
    
    # Фильтруем данные
    mask = np.isin(y_labels, valid_classes)
    X_filtered = X[mask]
    y_filtered = y[mask]
    
    # Переиндексируем метки
    y_labels_filtered = np.argmax(y_filtered, axis=1)
    label_encoder_new = LabelEncoder()
    y_labels_encoded = label_encoder_new.fit_transform(y_labels_filtered)
    y_filtered_categorical = to_categorical(y_labels_encoded)
    
    print(f"Данные после фильтрации: {X_filtered.shape[0]} примеров, {y_filtered_categorical.shape[1]} классов")
    
    return X_filtered, y_filtered_categorical, label_encoder_new

# Применяем фильтрацию
print("ФИЛЬТРАЦИЯ ДАННЫХ...")
X_train_filt, y_train_filt, label_encoder_filt = filter_top_classes(X_train, y_train, min_samples_per_class=3)
X_val_filt, y_val_filt, _ = filter_top_classes(X_val, y_val, min_samples_per_class=1)
X_test_filt, y_test_filt, _ = filter_top_classes(X_test, y_test, min_samples_per_class=1)

# Обновляем параметры
num_classes_filt = y_train_filt.shape[1]
print(f"Новое количество классов: {num_classes_filt}")


ФИЛЬТРАЦИЯ ДАННЫХ...
Всего классов: 1447
Классы с >= 3 примерами: 1021
Данные после фильтрации: 62241 примеров, 1021 классов
Всего классов: 1153
Классы с >= 1 примерами: 1153
Данные после фильтрации: 20960 примеров, 1153 классов
Всего классов: 1164
Классы с >= 1 примерами: 1164
Данные после фильтрации: 20961 примеров, 1164 классов
Новое количество классов: 1021


In [27]:
def create_model_for_filtered_data(vocab_size, sequence_length, num_classes):
    """Модель для отфильтрованных данных с меньшим количеством классов"""
    model = Sequential([
        Embedding(
            input_dim=vocab_size + 1,
            output_dim=32,
            input_length=sequence_length,
            name='embedding'
        ),
        tf.keras.layers.GlobalAveragePooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Создаем новую модель
model_filt = create_model_for_filtered_data(vocab_size, MAX_SEQUENCE_LENGTH, num_classes_filt)
print("Модель для отфильтрованных данных создана:")
model_filt.summary()


Модель для отфильтрованных данных создана:


c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
print("=== ТЕСТ НА ОТФИЛЬТРОВАННЫХ ДАННЫХ ===")

# Берем маленький батч из отфильтрованных данных
batch_size = min(16, len(X_train_filt))
X_test_batch = X_train_filt[:batch_size]
y_test_batch = y_train_filt[:batch_size]

print(f"Тестовый батч: {X_test_batch.shape}, {y_test_batch.shape}")

try:
    # Пробуем предсказание
    test_pred = model_filt.predict(X_test_batch, verbose=0)
    print(f"✓ Предсказание успешно! Output shape: {test_pred.shape}")
    
    # Пробуем обучение на одном батче
    print("Тестируем обучение на одном батче...")
    test_history = model_filt.fit(
        X_test_batch, y_test_batch,
        epochs=2,
        verbose=1,
        batch_size=8
    )
    print("✓ Обучение на тестовом батче успешно!")
    
except Exception as e:
    print(f"✗ Ошибка: {e}")


=== ТЕСТ НА ОТФИЛЬТРОВАННЫХ ДАННЫХ ===
Тестовый батч: (16, 30), (16, 1021)
✗ Ошибка: Graph execution error:

Detected at node sequential_1_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 640, in run_forever

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\

In [29]:
# Обучаем модель на всех отфильтрованных данных
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

print("=== ПОЛНОЕ ОБУЧЕНИЕ ===")
print(f"Обучающая выборка: {X_train_filt.shape[0]} примеров")
print(f"Валидационная выборка: {X_val_filt.shape[0]} примеров") 
print(f"Количество классов: {num_classes_filt}")

try:
    history = model_filt.fit(
        X_train_filt, y_train_filt,
        batch_size=16,
        epochs=50,
        validation_data=(X_val_filt, y_val_filt),
        callbacks=[early_stopping],
        verbose=1
    )
    
    print("✓ Модель успешно обучена!")
    model = model_filt
    
    # Сохраняем новую label_encoder
    label_encoder = label_encoder_filt
    
except Exception as e:
    print(f"✗ Ошибка при обучении: {e}")


=== ПОЛНОЕ ОБУЧЕНИЕ ===
Обучающая выборка: 62241 примеров
Валидационная выборка: 20960 примеров
Количество классов: 1021
Epoch 1/50
✗ Ошибка при обучении: Graph execution error:

Detected at node sequential_1_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 640, in run_forever

 

In [34]:
# Если предыдущие шаги не работают, создаем максимально простую модель
def create_ultra_simple_model(vocab_size, sequence_length, num_classes):
    """Ультра-простая модель"""
    model = Sequential([
        Embedding(vocab_size + 1, 8, input_length=sequence_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Пробуем ультра-простую модель
if 'model' not in locals():
    print("Пробуем ультра-простую модель...")
    ultra_model = create_ultra_simple_model(vocab_size, MAX_SEQUENCE_LENGTH, num_classes_filt)
    
    try:
        history = ultra_model.fit(
            X_train_filt[:100], y_train_filt[:100],  # Только 100 примеров
            batch_size=8,
            epochs=10,
            validation_data=(X_val_filt[:20], y_val_filt[:20]),
            verbose=1
        )
        model = ultra_model
        print("✓ Ультра-простая модель обучена!")
    except Exception as e:
        print(f"✗ Критическая ошибка: {e}")


In [42]:
ultra_model = create_ultra_simple_model(vocab_size, MAX_SEQUENCE_LENGTH, num_classes_filt)
    
try:
        history = ultra_model.fit(
            X_train_filt[:100], y_train_filt[:100],  # Только 100 примеров
            batch_size=8,
            epochs=10,
            validation_data=(X_val_filt[:20], y_val_filt[:20]),
            verbose=1
        )
        model = ultra_model
        print("✓ Ультра-простая модель обучена!")
except Exception as e:
        print(f"✗ Критическая ошибка: {e}")

c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
 1/13 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.0000e+00 - loss: 6.9284✗ Критическая ошибка: Graph execution error:

Detected at node sequential_5_1/embedding_3_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\ZubarevVV\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "c:\Users\ZubarevVV\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 640, in run_forever

  File "c:\Users\ZubarevVV\AppData\Local\Prog

In [33]:
def predict_category_new(text, model, tokenizer, label_encoder, max_sequence_length):
    """Обновленная функция предсказания для отфильтрованных классов"""
    processed_text = preprocess_text(text)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    prediction = model.predict(padded_sequence, verbose=0)
    predicted_class_idx = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)
    
    try:
        predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
        # Найдем оригинальное название класса
        original_class_name = "Предсказанный класс"
    except:
        predicted_class = f"Class_{predicted_class_idx}"
        original_class_name = "Неизвестный класс"
    
    return predicted_class, confidence, original_class_name

# Тестируем предсказание
if 'model' in locals():
    print("Тестируем предсказание...")
    test_text = "Масло трансмиссионное"
    category, confidence, original_name = predict_category_new(test_text, model, tokenizer, label_encoder, MAX_SEQUENCE_LENGTH)
    print(f"Текст: '{test_text}'")
    print(f"Предсказание: {category} (уверенность: {confidence:.3f})")


Тестируем предсказание...
Текст: 'Масло трансмиссионное'
Предсказание: Гайки (уверенность: 0.001)


In [36]:
# Диагностика проблемы
print("=== ДИАГНОСТИКА ПРОБЛЕМЫ ===")
print(f"Максимальный индекс в данных: {np.max(X_train)}")
print(f"Размер словаря (vocab_size): {vocab_size}")
print(f"MAX_NB_WORDS: {MAX_NB_WORDS}")

# Проверяем все данные на наличие индексов вне словаря
def check_indices(X_data, vocab_size, data_name):
    max_idx = np.max(X_data)
    min_idx = np.min(X_data)
    out_of_vocab = np.sum(X_data >= vocab_size)
    print(f"{data_name}: min={min_idx}, max={max_idx}, out_of_vocab={out_of_vocab}")

print("\nПроверка индексов:")
check_indices(X_train, vocab_size, "X_train")
check_indices(X_val, vocab_size, "X_val") 
check_indices(X_test, vocab_size, "X_test")


=== ДИАГНОСТИКА ПРОБЛЕМЫ ===
Максимальный индекс в данных: 4999
Размер словаря (vocab_size): 3000
MAX_NB_WORDS: 3000

Проверка индексов:
X_train: min=0, max=4999, out_of_vocab=41329
X_val: min=0, max=4999, out_of_vocab=13724
X_test: min=0, max=4999, out_of_vocab=13684


In [37]:
# Исправляем данные - обрезаем индексы до размера словаря
print("ИСПРАВЛЯЕМ ДАННЫЕ...")

# vocab_size должен быть равен MAX_NB_WORDS или количеству слов в tokenizer
actual_vocab_size = min(MAX_NB_WORDS, len(tokenizer.word_index) + 1)
print(f"Фактический размер словаря: {actual_vocab_size}")

# Обрезаем индексы во всех данных
X_train_fixed = np.clip(X_train, 0, actual_vocab_size - 1)
X_val_fixed = np.clip(X_val, 0, actual_vocab_size - 1) 
X_test_fixed = np.clip(X_test, 0, actual_vocab_size - 1)

print("После исправления:")
check_indices(X_train_fixed, actual_vocab_size, "X_train_fixed")
check_indices(X_val_fixed, actual_vocab_size, "X_val_fixed")
check_indices(X_test_fixed, actual_vocab_size, "X_test_fixed")

# Обновляем переменные
X_train = X_train_fixed
X_val = X_val_fixed
X_test = X_test_fixed
vocab_size = actual_vocab_size


ИСПРАВЛЯЕМ ДАННЫЕ...
Фактический размер словаря: 3000
После исправления:
X_train_fixed: min=0, max=2999, out_of_vocab=0
X_val_fixed: min=0, max=2999, out_of_vocab=0
X_test_fixed: min=0, max=2999, out_of_vocab=0


In [39]:
# Пересоздаем tokenizer с правильным num_words
print("ПЕРЕСОЗДАЕМ TOKENIZER...")

tokenizer_fixed = Tokenizer(num_words=MAX_NB_WORDS, filters='', lower=False)
tokenizer_fixed.fit_on_texts(texts)

# Пересоздаем последовательности с новым tokenizer
sequences_fixed = tokenizer_fixed.texts_to_sequences(texts)
X_fixed = pad_sequences(sequences_fixed, maxlen=MAX_SEQUENCE_LENGTH)

print(f"Новые данные: {X_fixed.shape}")
print(f"Новый максимальный индекс: {np.max(X_fixed)}")
print(f"Новый размер словаря: {len(tokenizer_fixed.word_index)}")

# Обновляем все данные
X = X_fixed


ПЕРЕСОЗДАЕМ TOKENIZER...
Новые данные: (104802, 30)
Новый максимальный индекс: 2999
Новый размер словаря: 77892


In [41]:
# Заново разделяем данные
indices = np.arange(len(X))

# Первое разделение: train+val / test
indices_temp, indices_test = train_test_split(
    indices, 
    test_size=0.2, 
    random_state=42, 
    stratify=labels
)

# Второе разделение: train / val
indices_train, indices_val = train_test_split(
    indices_temp,
    test_size=0.25,  # 0.2 * 0.25 = 0.05 от общего
    random_state=42,
    stratify=labels[indices_temp]
)

# Создаем выборки
X_train = X[indices_train]
X_val = X[indices_val] 
X_test = X[indices_test]
y_train = y[indices_train]
y_val = y[indices_val]
y_test = y[indices_test]

print("Исправленные размеры выборок:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# Проверяем индексы после разделения
print("\nПроверка индексов после разделения:")
check_indices(X_train, vocab_size, "X_train")
check_indices(X_val, vocab_size, "X_val")
check_indices(X_test, vocab_size, "X_test")


Исправленные размеры выборок:
X_train: (62880, 30), y_train: (62880, 1502)
X_val: (20961, 30), y_val: (20961, 1502)
X_test: (20961, 30), y_test: (20961, 1502)

Проверка индексов после разделения:
X_train: min=0, max=2999, out_of_vocab=0
X_val: min=0, max=2999, out_of_vocab=0
X_test: min=0, max=2999, out_of_vocab=0


In [43]:
# Создаем минимальную модель для теста
def create_safe_model(vocab_size, sequence_length, num_classes):
    """Безопасная модель с проверенными параметрами"""
    model = Sequential([
        Embedding(
            input_dim=vocab_size,  # Без +1, так как индексы от 0 до vocab_size-1
            output_dim=16,
            input_length=sequence_length,
            name='embedding'
        ),
        tf.keras.layers.GlobalAveragePooling1D(),
        Dense(8, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

# Создаем модель
safe_model = create_safe_model(vocab_size, MAX_SEQUENCE_LENGTH, num_classes)
print("Безопасная модель создана:")
safe_model.summary()


Безопасная модель создана:


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [44]:
print("=== ТЕСТ НА ИСПРАВЛЕННЫХ ДАННЫХ ===")

# Проверяем один пример
test_sample = X_train[:1]
print(f"Тестовый sample: {test_sample}")
print(f"Min index: {np.min(test_sample)}, Max index: {np.max(test_sample)}")

try:
    # Тестируем forward pass
    test_output = safe_model.predict(test_sample, verbose=0)
    print(f"✓ Forward pass успешен! Output shape: {test_output.shape}")
    
    # Тестируем на маленьком батче
    batch_size = min(8, len(X_train))
    X_batch = X_train[:batch_size]
    y_batch = y_train[:batch_size]
    
    print(f"Тестовый батч: {X_batch.shape}, {y_batch.shape}")
    
    history = safe_model.fit(
        X_batch, y_batch,
        epochs=2,
        batch_size=4,
        verbose=1
    )
    print("✓ Обучение на батче успешно!")
    
except Exception as e:
    print(f"✗ Ошибка: {e}")


=== ТЕСТ НА ИСПРАВЛЕННЫХ ДАННЫХ ===
Тестовый sample: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0 465   7  11 176 465   4  10  20 176  41]]
Min index: 0, Max index: 465
✓ Forward pass успешен! Output shape: (1, 1502)
Тестовый батч: (8, 30), (8, 1502)
Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.3125 
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1250 - loss: 7.3075
✓ Обучение на батче успешно!


In [48]:
# Продолжаем с исправленными данными, которые работают на тестовом батче
print("=== ПРОДОЛЖАЕМ С РАБОЧЕЙ ВЕРСИЕЙ ===")

# Используем данные, которые прошли тест на батче
X_train_working = X_train
X_val_working = X_val
X_test_working = X_test
y_train_working = y_train
y_val_working = y_val
y_test_working = y_test

print(f"Рабочие данные:")
print(f"X_train: {X_train_working.shape}")
print(f"X_val: {X_val_working.shape}")
print(f"X_test: {X_test_working.shape}")


=== ПРОДОЛЖАЕМ С РАБОЧЕЙ ВЕРСИЕЙ ===
Рабочие данные:
X_train: (62880, 30)
X_val: (20961, 30)
X_test: (20961, 30)


In [51]:
# Функция для фильтрации классов с малым количеством примеров
def filter_classes_by_count(X, y, min_samples=2):
    """Оставляем только классы с минимальным количеством примеров"""
    y_labels = np.argmax(y, axis=1)
    unique_classes, class_counts = np.unique(y_labels, return_counts=True)
    
    # Находим классы с достаточным количеством примеров
    valid_classes = unique_classes[class_counts >= min_samples]
    
    print(f"Всего классов: {len(unique_classes)}")
    print(f"Классы с >= {min_samples} примерами: {len(valid_classes)}")
    print(f"Распределение: {dict(zip(unique_classes, class_counts))}")
    
    # Создаем маску для валидных классов
    mask = np.isin(y_labels, valid_classes)
    
    if np.sum(mask) == 0:
        print("Предупреждение: нет классов с достаточным количеством примеров")
        # Берем классы с максимальным количеством примеров
        top_classes = unique_classes[np.argsort(class_counts)[-5:]]  # Топ-5 классов
        mask = np.isin(y_labels, top_classes)
        print(f"Используем топ-5 классов: {top_classes}")
    
    X_filtered = X[mask]
    y_filtered = y[mask]
    
    # Переиндексируем метки от 0
    y_labels_filtered = np.argmax(y_filtered, axis=1)
    label_encoder_new = LabelEncoder()
    y_labels_encoded = label_encoder_new.fit_transform(y_labels_filtered)
    y_filtered_categorical = to_categorical(y_labels_encoded)
    
    print(f"После фильтрации: {X_filtered.shape[0]} примеров, {y_filtered_categorical.shape[1]} классов")
    
    return X_filtered, y_filtered_categorical, label_encoder_new

# Применяем фильтрацию к обучающим данным
print("ФИЛЬТРАЦИЯ ОБУЧАЮЩИХ ДАННЫХ:")
X_train_filt, y_train_filt, label_encoder_train = filter_classes_by_count(X_train_working, y_train_working, min_samples=2)

# Для валидации и теста используем те же классы, что и в train
def filter_to_train_classes(X, y, train_label_encoder):
    """Фильтруем данные чтобы содержать только классы из обучающей выборки"""
    y_labels = np.argmax(y, axis=1)
    
    # Получаем классы из обучающей выборки
    train_classes = train_label_encoder.classes_
    
    # Создаем маску
    mask = np.isin(y_labels, train_classes)
    X_filtered = X[mask]
    y_filtered = y[mask]
    
    # Переиндексируем согласно обучающему encoder
    y_labels_filtered = np.argmax(y_filtered, axis=1)
    y_labels_encoded = train_label_encoder.transform(y_labels_filtered)
    y_filtered_categorical = to_categorical(y_labels_encoded)
    
    print(f"Отфильтровано: {X_filtered.shape[0]} примеров")
    
    return X_filtered, y_filtered_categorical

print("\nФИЛЬТРАЦИЯ ВАЛИДАЦИОННЫХ ДАННЫХ:")
X_val_filt, y_val_filt = filter_to_train_classes(X_val_working, y_val_working, label_encoder_train)

print("\nФИЛЬТРАЦИЯ ТЕСТОВЫХ ДАННЫХ:")
X_test_filt, y_test_filt = filter_to_train_classes(X_test_working, y_test_working, label_encoder_train)

# Обновляем количество классов
num_classes_filt = y_train_filt.shape[1]
print(f"\nИтоговое количество классов: {num_classes_filt}")


ФИЛЬТРАЦИЯ ОБУЧАЮЩИХ ДАННЫХ:
Всего классов: 1502
Классы с >= 2 примерами: 1414
Распределение: {np.int64(0): np.int64(1), np.int64(1): np.int64(5), np.int64(2): np.int64(4), np.int64(3): np.int64(3), np.int64(4): np.int64(2), np.int64(5): np.int64(7), np.int64(6): np.int64(2), np.int64(7): np.int64(8), np.int64(8): np.int64(2), np.int64(9): np.int64(4), np.int64(10): np.int64(2), np.int64(11): np.int64(40), np.int64(12): np.int64(2), np.int64(13): np.int64(10), np.int64(14): np.int64(122), np.int64(15): np.int64(31), np.int64(16): np.int64(4), np.int64(17): np.int64(18), np.int64(18): np.int64(8), np.int64(19): np.int64(4), np.int64(20): np.int64(2), np.int64(21): np.int64(1), np.int64(22): np.int64(7), np.int64(23): np.int64(10), np.int64(24): np.int64(63), np.int64(25): np.int64(2), np.int64(26): np.int64(58), np.int64(27): np.int64(4), np.int64(28): np.int64(2), np.int64(29): np.int64(25), np.int64(30): np.int64(9), np.int64(31): np.int64(52), np.int64(32): np.int64(2), np.int64(33):

MemoryError: Unable to allocate 720. MiB for an array with shape (62792, 1502) and data type float64